Todo:

3. Prepare the whole pipeline
    1. Data augmentation
        - First take the unaugmented original dataset and proceed
        -  Augment the data with a predefined seed for each of the following techniques: rotation, flipping, contrast, brightness change, random erasing
    2. Choose hyperparameters
        - For the hyper-parameters related to the training process we chose batch size, learning rate, and number of epochs
        - For the hyper-parameters related to Regularization we decided to use L2 Regularization (Weight Decay) and Dropout Rate
    3. Train each of the prepared models on the augmented dataset for a chosen augmentation technique
    4. Test and collect data regarding models’ performance on the augmented dataset
    5. Repeat several times (>=3) from 3.
    6. Choose different values for hyperparameters and start from 3.
    7. Choose the next augmentation technique and start from 2.
    8. Repeat the process starting from 1. several times (>=3) with a different seed each time

In [9]:
import os
from enum import Enum
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization
from efficientnet.tfkeras import EfficientNetB0
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.preprocessing.image import ImageDataGenerator

ModuleNotFoundError: No module named 'distutils'

## Move 5400 images from each class from valid to train
 - There is a safety check, if it has been already done it won't do it again :)

In [19]:
rootdir = './Dataset/valid'

for subdir, dirs, files in os.walk(rootdir):
    if subdir != rootdir:
        for subsubdir, subdirs, files in os.walk(subdir):
            if len(files) < 5400:
                break;
            for i in range(5400):
                os.rename(os.path.join(os.path.join("./Dataset/valid",os.path.basename(subsubdir)),files[i]), os.path.join(os.path.join("./Dataset/train",os.path.basename(subsubdir)),files[i]))

## Pipeline

Classes

In [10]:
class AugmentationTechnique(Enum):
    NoAugmentation = 0
    Rotation = 1
    Flipping = 2
    Contrast = 3
    Brightness = 4
    RandomErasing = 5

class ModelType(Enum):
    MobileNet =1
    EfficientNet = 2

class Model:
    def fit(self,batch_size,epochs,train_data_generator,valid_data_generator):
        pass
    def predict(self):
        pass



# TODO: implement those classes for models
def CustomMobileNetModel(Model):
    def __init__(self,optimizer,loss,metrics):
        # Load MobileNetV3Large without top classification layer
        base_model = MobileNetV3Large(include_top=False, weights='imagenet', input_shape=(32, 32, 3))

        # Freeze the base model layers
        base_model.trainable = False

        # Add additional layers on top of MobileNetV3Large
        model = tf.keras.Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')  # Output layer with size 10 for classification
        ])
        # Compile the model
        model.compile(optimizer, loss, metrics)
        self.model=model

    def fit(self,batch_size,epochs,train_data_generator,valid_data_generator):
        self.model.fit(
        train_data_generator,
        steps_per_epoch=batch_size,
        epochs=epochs,
        validation_data=valid_data_generator,
        validation_steps=batch_size
        )
    def predict(self,test_data_generator):
        self.model.predict(test_data_generator, steps=len(test_data_generator))

def CustomEfficientNetModel(Model):
    def __init__(self):
        # Load MobileNetV3Large without the top classification layer
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
        
        # Freeze the base model layers
        base_model.trainable = False
        # Add additional layers on top of EfficientNet
        model = tf.keras.Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')  # Output layer with size 10 for classification
        ])
        # Compile the model
        optimizer = Adam(learning_rate=0.001)
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        self.model=model
    def fit(self,batch_size,epochs,train_data_generator,valid_data_generator):
        self.model.fit(
        train_data_generator,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=valid_data_generator,
        )
    def predict(self,test_data_generator):
        predictions = self.model.predict(test_data_generator)


Helper functions

In [3]:

def createModel(modelType, optimizer, loss, metrics)->Model:
    match modelType:
        # TODO: Implement the creation of these models and then return the model
        case ModelType.MobileNet:
            return CustomMobileNetModel(optimizer,loss,metrics)
        case ModelType.EfficientNet:
            return NotImplementedError

def augumentData(data, technique, seed):
    match technique:
        case AugmentationTechnique.Rotation:
            print("Hello")

    return data

def getAccuracy(y_result, y_test):
    correct_amount =0 
    for i, result in enumerate(y_result):
        if result == y_test[i]:
            correct_amount+=1
    return correct_amount/len(y_test)

Arrays containing different hyper-parameter values

TODO: Handle hyperparameters related to regularization

In [13]:
# training process
batchSizes =[]
learningRates = []
numberOfEpochs =[]

# regularization



# augmentation
augmentationTechniques =[AugmentationTechnique.NoAugmentation,AugmentationTechnique.Rotation,AugmentationTechnique.Flipping,AugmentationTechnique.Contrast,AugmentationTechnique.Brightness,AugmentationTechnique.RandomErasing]


REPETITIONS = 3
seeds = [123,42,56]

Main pipeline loop

TODO: repeat the experiments 3 times, different seed each time. I want to first test if it works once

In [ ]:
def performExperiment(modelType,X,y, X_test, y_test):
    results = []
    for batchSize in batchSizes:
        for learningRate in learningRates:
            for epochNumber in numberOfEpochs:
                for augmentation in augmentationTechniques:
                    augX = augumentData(X,augmentation,seeds[0])
                    # create the model
                    model = createModel(modelType,optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
                    # train the model 
                    model.fit(augX, y,batch_size=batchSize,epochs=epochNumber)

                    # get accuracy
                    y_pred = model.predict(X_test)
                    accuracy = getAccuracy(y_pred,y_test)

                    # append to results
                    results.append({'accuracy':accuracy,'augmentation': augmentation,'batchSize':batchSize,'learningRate':learningRate,'numberOfEpochs':numberOfEpochs})
        

# UNUSED

In [ ]:


class HyperParameter(Enum):
    BatchSize=1
    LearningRate =2
    NumberOfEpochs =3

class HyperParameters:
    def __init__(self, batchSizes, learningRates, numberOfEpochs):
        self.batchSizes=batchSizes
        self.learningRates=learningRates
        self.numberOfEpochs=numberOfEpochs
        self.currentIndex =0
    
    def getNextHyperParameter(self):
        if self.currentIndex<len(self.batchSizes):
            return self.batchSizes[self.currentIndex], HyperParameter.BatchSize
        elif self.currentIndex<(len(self.batchSizes+self.learningRates)):
            return self.learningRates[self.currentIndex-len(self.batchSizes)], HyperParameter.LearningRate
        elif self.currentIndex<(len(self.batchSizes)+len(self.learningRates)+len(self.numberOfEpochs)):
            return self.num
        

